In [1]:
pip install alpaca-trade-api


  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached PyYAML-6.0.1-cp312-cp312-win_amd64.whl.metadata (2.1 kB)
Using cached PyYAML-6.0.1-cp312-cp312-win_amd64.whl (138 kB)
  Created wheel for msgpack: filename=msgpack-1.0.3-cp312-cp312-win_amd64.whl size=15816 sha256=3a67c2b67fdf4b8ccb03bad2eadad1f3879e36d37aa75ed3f70c68f3e9954201
  Stored in directory: c:\users\sneh_\appdata\

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yfinance 0.2.61 requires websockets>=13.0, but you have websockets 10.4 which is incompatible.

[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from alpaca_trade_api.rest import REST, TimeFrame
import time

In [6]:
API_KEY = 'PK5ZC5R1JKB7RGDRPVPR'
API_SECRET = 'LscrhAed3acH0tVkZUgIkqRaCdDxi9FQrarXs2of'
BASE_URL = 'https://paper-api.alpaca.markets'

api = REST(API_KEY, API_SECRET, BASE_URL)

In [7]:
account = api.get_account()
print("Account status:", account.status)
print("Cash:", account.cash)
print("Buying Power:", account.buying_power)

Account status: ACTIVE
Cash: 100000
Buying Power: 200000


In [13]:
# Submit a Market Order with optional stop-loss and take-profit
def place_bracket_order(symbol, qty, take_profit_pct=1.05, stop_loss_pct=0.97):
    current_price = float(api.get_latest_trade(symbol).price)
    take_profit_price = round(current_price * take_profit_pct, 2)
    stop_loss_price = round(current_price * stop_loss_pct, 2)

    order = api.submit_order(
        symbol=symbol,
        qty=qty,
        side='buy',
        type='market',
        time_in_force='gtc',
        order_class='bracket',
        take_profit={'limit_price': take_profit_price},
        stop_loss={'stop_price': stop_loss_price}
    )
    print(f"Bracket order submitted for {symbol} @ ${current_price}")
    print(f"TP: ${take_profit_price}, SL: ${stop_loss_price}")
    return order

In [14]:
# Submit a Limit Order
def place_limit_order(symbol, qty, limit_price):
    order = api.submit_order(
        symbol=symbol,
        qty=qty,
        side='buy',
        type='limit',
        limit_price=limit_price,
        time_in_force='gtc'
    )
    print(f"Limit order for {symbol} @ ${limit_price} placed.")
    return order

In [15]:
# Track Portfolio
def track_portfolio():
    print("\n📊 Current Portfolio:")
    positions = api.list_positions()
    for pos in positions:
        print(f"{pos.symbol}: {pos.qty} shares, Market Value: ${pos.market_value}, Unrealized P/L: ${pos.unrealized_pl}")
    
    account = api.get_account()
    print(f"Portfolio Value: ${account.portfolio_value}")
    print(f"Cash: ${account.cash}\n")

In [16]:
# Example Usage
symbol = 'AAPL'
qty = 1
place_bracket_order(symbol, qty)

Bracket order submitted for AAPL @ $209.94
TP: $220.44, SL: $203.64


Order({   'asset_class': 'us_equity',
    'asset_id': 'b0b6dd9d-8b9b-48a9-ba46-b9d54906e415',
    'canceled_at': None,
    'client_order_id': '1ed74923-6e33-4203-9cac-95f02a609cee',
    'created_at': '2025-07-07T22:40:32.072226606Z',
    'expired_at': None,
    'expires_at': '2025-10-06T20:00:00Z',
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': 'cdbf7538-1c55-4e6c-a2df-7d88b1fd5fa9',
    'legs': [   {   'asset_class': 'us_equity',
                    'asset_id': 'b0b6dd9d-8b9b-48a9-ba46-b9d54906e415',
                    'canceled_at': None,
                    'client_order_id': '2e84b6f8-7d8f-4c05-a306-08945314ca1e',
                    'created_at': '2025-07-07T22:40:32.072226606Z',
                    'expired_at': None,
                    'expires_at': '2025-10-06T20:00:00Z',
                    'extended_hours': False,
                    'failed_at': None,
               

In [17]:
# Optional delay to allow order processing before tracking
time.sleep(5)
track_portfolio()


📊 Current Portfolio:
Portfolio Value: $100000
Cash: $100000

